<a href="https://colab.research.google.com/github/AndrewFatula/Seq2seq_model_using_Google_Cloud_TPU/blob/master/wise_bot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import string
import os
import sys
import time
from nltk.tokenize import TweetTokenizer
import tensorflow as tf
from collections import Counter
from matplotlib import pyplot as plt
from nltk.translate import bleu_score
from keras import regularizers
from copy import deepcopy as dc
import tensorboardcolab as tb
print(tf.__version__)

2.0.0


Using TensorFlow backend.


In [0]:
!pip install --upgrade auth
!pip uninstall grpcio
!pip uninstall tensorflow
!pip install grpcio==1.24.3
!pip install tensorflow==2.0.0

In order to use google cloud TPUv1 with distributed TPU strategy in eager execution mode we need to reinstall tensorflow 2.0.0 and grpcio 1.24.3

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Authentication for transfering data from Google Drive

In [0]:
downloaded = drive.CreateFile({'id':'17XFYmocGDatGd40cAijhCOkI_zRXN5Mi'}) # all phrases in movies
downloaded.GetContentFile('wise_quotes.txt') 
downloaded = drive.CreateFile({'id':'1Gn8fDg8Yhv8k0AN4XmKt8C9om_RgvyTu'}) # all phrases in movies
downloaded.GetContentFile('wise_quotes2.txt') 
downloaded = drive.CreateFile({'id':'1cibzj_h-IDmoNCYcv9wiOjHFh34XhqzD'}) # all phrases in movies
downloaded.GetContentFile('wise_quotes3.txt') 
downloaded = drive.CreateFile({'id':'1r5SQab9h4f-nyXu8rt-et0iiGdSEJ6rm'}) # all phrases in movies
downloaded.GetContentFile('quotes.csv') 
downloaded = drive.CreateFile({'id':'1yNvbktnsPaqG7OkEfhyxV-yrI2TiP-PD'}) # all phrases in movies
downloaded.GetContentFile('bible_quotes.csv') 


Transering needed data from Google drive

In [0]:



if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address iis', tpu_address)


cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
    tpu=tpu_address)
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)

print ('Number of devices: {}'.format(tpu_strategy.num_replicas_in_sync))


TPU address iis grpc://10.18.84.2:8470
INFO:tensorflow:Initializing the TPU system: 10.18.84.2:8470


INFO:tensorflow:Initializing the TPU system: 10.18.84.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of devices: 8


Initializing available TPU devices and defining TPU_Distributed_strategy

In [0]:
MAX_QUOTE_LENGTH = 30
MIN_QUOTE_LENGTH = 5
MIN_TOKEN_FREQ = 5
QUOTES_FILES = ["wise_quotes.txt", "wise_quotes2.txt", "wise_quotes3.txt", "quotes.csv", "bible_quotes.csv"]


def read_quotes(quotes_files):

    """ Function that reads all the data  (philosophical quotes from txt files, 
        bible quotes from csv file and philosophical quotes from csv file)
    """

    print("Reading quotes...")

    lengths = []
    tokenized_lines = []
    freq_count = Counter()
    tokenizer = TweetTokenizer()

    #reading data from csv file
    additional_quotes = list(pd.read_csv(quotes_files[3], sep = ",")["quote"])
    bible_quotes = list(pd.read_csv(quotes_files[4], sep = ",")["english"])

    #reading data from txt files
    for quotes_file in quotes_files[:3]:
        with open(quotes_file, "r") as quotes_lines:
            for line in quotes_lines:
                line = line.lower()
                tokenized = tokenizer.tokenize(line)
                if len(tokenized) < MAX_QUOTE_LENGTH + 1 and len(tokenized) > MIN_TOKEN_FREQ:
                    freq_count.update(tokenized)
                    tokenized_lines.append(tokenized)

    #tokenization of all readed string lines
    for line in additional_quotes + bible_quotes:
        line = line.lower()
        line = line.strip('"')
        line = line.strip("'")
        tokenized = tokenizer.tokenize(line)
        if len(tokenized) < MAX_QUOTE_LENGTH + 1 and len(tokenized) > MIN_TOKEN_FREQ:
            freq_count.update(tokenized)
            tokenized_lines.append(tokenized)

    #construction of words dictionary
    word_set = list(map( lambda x: '+' + x[0] if x[1] >= MIN_TOKEN_FREQ else '-' + x[0], freq_count.items() ))
    word_dict = {"EMPTY":0, "BEGIN":1, "END" : 2}
    i = 3
    for word in word_set:
        if word[0] == "+":
            word_dict[word[1:]] = i
            i+=1

    #filtering all tokenized quotes (leaving ones with known words)
    quotes_tokens = []
    for quote in tokenized_lines:
        tokens = []
        for word in quote:
            if word in word_dict.keys():
                tokens.append(word_dict[word])
            else:
                tokens.append(word_dict["EMPTY"])
        if not word_dict["EMPTY"] in tokens:
            tokens = [word_dict["BEGIN"]] + tokens + [word_dict["END"]]
            quotes_tokens.append(tokens)
            lengths.append(len(tokens))

    print("Quotes are read and tokenized...")
    return quotes_tokens, word_dict, lengths


In [0]:




tokenized_lines, word_dict, true_lengths = read_quotes(QUOTES_FILES)

quotes_width = int(max(true_lengths))
sparse_quotes = np.zeros((len(true_lengths), quotes_width))
quotes_length = len(true_lengths) 

inverse_word_dict = {}
for key in word_dict.keys():
    inverse_word_dict[word_dict[key]] = key

#padding each sequence in x_data and y_data
for i in range(quotes_length):
    sparse_quotes[i, : true_lengths[i]] = np.array(tokenized_lines[i])

unique_tokens = np.unique(sparse_quotes)

transfer_tokens = {}
transfer_dict = {}
for i in range(len(unique_tokens)):
    transfer_tokens[unique_tokens[i]] = i
    transfer_dict[inverse_word_dict[unique_tokens[i]]] = i

inverse_word_dict = {}

for key in transfer_dict.keys():
    inverse_word_dict[transfer_dict[key]] = key

word_dict = transfer_dict

for i in range(quotes_length):
    for j in range(quotes_width):
        sparse_quotes[i,j] = transfer_tokens[sparse_quotes[i,j]]


true_lengths = tf.convert_to_tensor(true_lengths, dtype = tf.float32)
sparse_quotes = tf.convert_to_tensor(sparse_quotes, dtype = tf.float32)

print("Dictionary_size: ", len(word_dict))
print("N of tokenized_quotes:", len(tokenized_lines))


Reading quotes...
Quotes are read and tokenized...
Dictionary_size:  22134
N of tokenized_quotes: 256487


Code above padds all word tokens and converts them to tensorflow arrays

In [0]:
with tpu_strategy.scope():


    class Phrase_model(tf.keras.Model):

        ''' inherited from tf.keras.model class simle seq2seq model class with one embedding layer, 2 recurrent layers and two dense layers  '''

        def __init__( self, hidden_size, emb_size, random_size, quote_len,
                      word_dict, inverse_word_dict ):

            super(Phrase_model, self).__init__()
            self.dict_size = len(word_dict)
            self.hidden_size = hidden_size
            self.emb_size = emb_size
            self.quote_len = quote_len
            self.word_dict = word_dict
            self.inverse_word_dict = inverse_word_dict

            self.emb_layer = tf.keras.layers.Embedding(self.dict_size, emb_size)
    
            self.generator = tf.keras.layers.LSTM( units = hidden_size, recurrent_activation = "sigmoid",
                                               kernel_regularizer=regularizers.l2(0.003), 
                                               recurrent_regularizer=regularizers.l2(0.003),
                                               bias_regularizer=regularizers.l2(0.003),
                                               dropout = 0.05, recurrent_dropout = 0.01, 
                                               return_state = True, return_sequences = True, unroll = True )
            
            self.interpreter = tf.keras.layers.Dense(self.dict_size)

            
        def generate_sequence_teacher(self, true_sequences):

            ''' method for training seq2seq with teacher 
                witch takes as argument encoded hidden state from x_input sequence
            '''
            emb_sequences = self.emb_layer(true_sequences)

            output, _, _ = self.generator(emb_sequences)
            #output, _ = self.generator(emb_sequences)
            return self.interpreter(output)





In code above you can see Pharse_model class used for training Hidden Markov Chain, subclassed from tf.keras.Model class.

In [0]:
with tpu_strategy.scope():

    def compute_loss( labels, generated_output, semihot_sequences ):

        ''' cross_entropy_loss function for distributed strategy'''

        per_example_loss = loss_object(labels, generated_output) * semihot_sequences
        return tf.nn.compute_average_loss(per_example_loss , global_batch_size=BATCH_SIZE)


    @tf.function
    def teacher_train_step(inputs):

        ''' train_step1 fuction for crossentropy teacher training method 
            of training LSTM cell, that will be crucial in construction Hidden Markov Chain
        '''

        true_sequences = inputs[0]
        true_semihot = inputs[1]

        with tf.GradientTape() as tape:
            generated_output = seq2seq.generate_sequence_teacher(true_sequences[:, :-1])
            loss = compute_loss(true_sequences[:, 1:], generated_output, true_semihot[:, 1:])
        gradients = tape.gradient(loss, seq2seq.trainable_variables)
        optimizer1.apply_gradients(zip(gradients, seq2seq.trainable_variables))
        return loss



    def distributed_teacher_train_step(dataset_inputs):

        ''' function that applies distriuted TPU strategy to train_step2 function'''

        per_replica_losses = tpu_strategy.experimental_run_v2(teacher_train_step,
                                                            args=(dataset_inputs,))
        return tpu_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                                                    axis=None)




In code above are defined distributed_train_step function decorated with @tf.fucntion for construction static computation graph in order to gain all benefits of using Google cloud TPU V1 and distrivuted tpu_strategy. 

In [0]:
with tpu_strategy.scope():

    def main_training_loop(train_dist_dataset, teacher_ptob, crossentropy_losses):

        ''' training function for crossentropy training loop
        '''
        for x in train_dist_dataset:
            loss = distributed_teacher_train_step(x)
            crossentropy_losses.append(loss)

            

Above is defined function to perform training loop

In [0]:
EMBEDDING_SIZE = 128
RANDOM_SIZE = 128
HIDDEN_SIZE = 2048
BATCH_SIZE = 8192

N_EPOCHS = 100

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)

with tpu_strategy.scope():

	dict_size = len(word_dict)
	seq2seq = Phrase_model( HIDDEN_SIZE, EMBEDDING_SIZE, RANDOM_SIZE, MAX_QUOTE_LENGTH + 2,
	                word_dict, inverse_word_dict )


	train_length = np.shape(sparse_quotes)[0]
	train_steps = int(train_length/BATCH_SIZE)

	mean_crossentropy_losses = []
	mean_discriminator_losses = []
	mean_generator_losses = []

	LEARNING_RATE = tf.keras.optimizers.schedules.ExponentialDecay(0.001, decay_steps = train_steps, decay_rate = 0.95, staircase= True)  
	loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction=tf.keras.losses.Reduction.NONE)
	optimizer1 = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)



	print("start training ... ")
	for epoch in range(N_EPOCHS):
		epoch_start = time.localtime(time.time())

		
		crossentropy_losses = []

		true_semihot = tf.ones((train_length, MAX_QUOTE_LENGTH + 2), dtype = tf.float32) * tf.range(1, MAX_QUOTE_LENGTH + 3, dtype = tf.float32)[None, :]
		true_semihot = tf.where(true_semihot <= true_lengths[ :, None], 1., 0.)
		true_semihot = tf.cast(true_semihot, tf.float32)

		dataset = tf.data.Dataset.from_tensor_slices((sparse_quotes, true_semihot)).shuffle(150000).batch(BATCH_SIZE, drop_remainder = True) 
		train_dist_dataset = tpu_strategy.experimental_distribute_dataset(dataset)

		main_training_loop(train_dist_dataset, teacher_prob, crossentropy_losses)

		epoch_end = time.localtime(time.time())
		start_in_sec = epoch_start[3]*3600 + epoch_start[4]*60 + epoch_start[5]
		end_in_sec = epoch_end[3]*3600 + epoch_end[4]*60 + epoch_end[5]
		epoch_time = end_in_sec - start_in_sec


		crossentropy_loss = np.mean(crossentropy_losses)
		mean_crossentropy_losses.append(crossentropy_loss)

		print( "Epoch: ", epoch, ", loss:", crossentropy_loss, "Teacher_prob: ", teacher_prob,
					", time: ", epoch_time ," sec "  )

		



		


				
		



start training ... 
Epoch:  0 , loss: 15.786491 Teacher_prob:  1 , time:  38  sec 
Epoch:  1 , loss: 15.142873 Teacher_prob:  1 , time:  23  sec 
Epoch:  2 , loss: 15.01017 Teacher_prob:  1 , time:  23  sec 
Epoch:  3 , loss: 14.937063 Teacher_prob:  1 , time:  23  sec 
Epoch:  4 , loss: 14.883511 Teacher_prob:  1 , time:  23  sec 
Epoch:  5 , loss: 14.841157 Teacher_prob:  1 , time:  23  sec 
Epoch:  6 , loss: 14.804003 Teacher_prob:  1 , time:  22  sec 
Epoch:  7 , loss: 14.77085 Teacher_prob:  1 , time:  23  sec 
Epoch:  8 , loss: 14.73738 Teacher_prob:  1 , time:  23  sec 
Epoch:  9 , loss: 14.707952 Teacher_prob:  1 , time:  23  sec 
Epoch:  10 , loss: 14.680026 Teacher_prob:  1 , time:  23  sec 
Epoch:  11 , loss: 14.655442 Teacher_prob:  1 , time:  22  sec 
Epoch:  12 , loss: 14.629939 Teacher_prob:  1 , time:  23  sec 
Epoch:  13 , loss: 14.608069 Teacher_prob:  1 , time:  23  sec 
Epoch:  14 , loss: 14.586387 Teacher_prob:  1 , time:  23  sec 
Epoch:  15 , loss: 14.566985 Teac

KeyboardInterrupt: ignored

Above is defined main training Cycle for training Hidden Markov Chain on philosophical and bible quotes. 

In [0]:
with tf.device("cpu"):

    
    generated_sequence, _ = seq2seq.generate_sequence(30,1)

    generated_sequence = generated_sequence.numpy()

    for idx in range(30):
        print("\n")
        text_recreated = ""
        for token in generated_sequence[idx]:
            if token == 2:
                break
            text_recreated += inverse_word_dict[token] + " "

        print(text_recreated)




he who has a why to live can bear almost any how . 


we are all born for love . it is the principle of existence , and its only end . 


it's not the end of the road , it's just the end of the road work . 


what is the point of having a civilization , if we do not practice being civilized ! 


i am a great believer in luck , and i find the harder i work the more i have of it . 


the only thing that makes life possible is permanent , intolerable uncertainty : not knowing what comes next . 


we are all born for love . it is the principle of existence , and its only end . 


every man is a damn fool for at least five minutes every day wisdom consists in not exceeding the limit . 


pain is a part of life . i hated that fact . i would be much happier without it but then it wouldn't be my life . 


everybody has a dream , to make that dream a reality , one must work hard . 


eternity is a long time to spend alone , without others of your kind . 


it's not the end of the road , it's 

In [0]:
with tf.device("cpu"):

    
    generated_sequence, _ = seq2seq.generate_sequence(30,7)

    generated_sequence = generated_sequence.numpy()

    for idx in range(30):
        print("\n")
        text_recreated = ""
        for token in generated_sequence[idx]:
            if token == 2:
                break
            text_recreated += inverse_word_dict[token] + " "

        print(text_recreated)

    



i am a great believer in luck , and i find the harder i work the more i have of it . 


travel is the best teacher . the only way to an open mind is by taking a plane out into the open world . 


that was the thing about weddings : they forced family members to deal with one another , like it or not . 


you can ’ t get the taste of winning in a running competition with a turtle ! 


no one can be a better you than you . therefore , striving continuously to become a better you is the only path to your greatness . 


my dad was a big frank zappa fan , so i remember listening to a lot of frank zappa . girls do not like frank zappa . 


it is not the size of the ring , but the size of the love . 


sexy can really lead you in a small one . 


i'm not a big fan of talking about dying . and then i make a movie where i kill everybody . 


to be a success , follow your obsessions obsessively . 


if you want to be happy , be . 


my father was a man , and i know the sex pretty well . 


you

In [0]:
phrases = [
            "it's better to have more",
            "man",
            "intelligence",
            "sounds are like",
            "strength",
            "be or not to be",
            "how to convince someone", 
            "set of words", 
            "not a friend",
            "how to act", 
            "character",
            "rude",
            "seriousness",
            "wise man",
            "wise"
          ]

def end_phrase(phrase):
    phrase = ["BEGIN"] + phrase.split(" ")
    phrase_tokens = []
    hidden_h = tf.zeros((1, HIDDEN_SIZE), dtype = tf.float32)
    hidden_c = tf.zeros((1, HIDDEN_SIZE), dtype = tf.float32)

    for word in phrase:
        phrase_tokens.append(word_dict[word])

    total_output = phrase_tokens[1:]

    for token in phrase_tokens:
        current_emb = seq2seq.emb_layer(tf.convert_to_tensor([[token]], dtype = tf.float32))
        current_output, hidden_h, hidden_c = seq2seq.generator(current_emb, [hidden_h, hidden_c])
        current_distribution = seq2seq.interpreter(current_output)

    generated_token = tf.argmax(current_distribution, axis = -1)
    total_output.append(generated_token.numpy()[0,0])
    current_emb = seq2seq.emb_layer(generated_token)
    
    for _ in range(30):
        current_output, hidden_h, hidden_c = seq2seq.generator(current_emb, [hidden_h, hidden_c])
        current_distribution = seq2seq.interpreter(current_output)
        generated_token  = tf.argmax(current_distribution, axis = -1)
        current_emb = seq2seq.emb_layer(generated_token)
        
        if not ( generated_token.numpy()[0,0] == 2 or generated_token.numpy()[0,0] == 0):
            total_output.append(generated_token.numpy()[0,0])
        else:
            break
    
    output_phrase = ""

    for token in total_output:
        output_phrase += inverse_word_dict[token] + " "

    print(output_phrase)


for phrase in phrases:
    end_phrase(phrase)








it's better to have more life than offer and clear when i write about ' and ' will , not even a few can afford to be ! 
man is a social animal . 
intelligence is the ability to adapt to change . 
sounds are like going ' in the end . 
strength is not determined by the looks , but action . 
be or not to be rich and believe in your ability to write them out loud and do the rest of your dreams . 
how to convince someone else ’ s name and what is ‘ matter with them ! 
set of words help but seek to manifest . 
not a friend goes by the justice industry . 
how to act : try to keep quiet . 
character is the ability to carry out a good resolution long after the excitement of the moment has passed . 
rude and wrong , it's right to be positive . 
seriousness is a series of which i even the young knows two thousand words , and that's now . 
wise man loses more than pleasure , for the best reason he is trying to make living choices . 
wise men speak because they have something to say 


In [0]:
beginnings = sparse_quotes.numpy()[:,1]

In [0]:
freq_count = Counter()
freq_count.update(beginnings)
freq_count = list(freq_count.items())
freq_count.sort(key = lambda x: x[1], reverse = True)


begin_tokens = [freq_count[i][0] for i in range(1500)]

In [0]:
print(begin_tokens)

[63.0, 77.0, 25.0, 140.0, 41.0, 61.0, 29.0, 66.0, 103.0, 67.0, 28.0, 70.0, 235.0, 11.0, 75.0, 39.0, 20.0, 217.0, 164.0, 369.0, 146.0, 170.0, 377.0, 166.0, 14.0, 46.0, 292.0, 206.0, 13.0, 350.0, 58.0, 86.0, 69.0, 256.0, 18.0, 366.0, 273.0, 74.0, 306.0, 261.0, 510.0, 351.0, 938.0, 55.0, 896.0, 173.0, 180.0, 159.0, 691.0, 254.0, 420.0, 129.0, 95.0, 309.0, 7.0, 104.0, 411.0, 222.0, 107.0, 59.0, 117.0, 1579.0, 536.0, 228.0, 333.0, 73.0, 1329.0, 82.0, 19.0, 27.0, 262.0, 472.0, 234.0, 33.0, 272.0, 44.0, 83.0, 1030.0, 944.0, 290.0, 153.0, 141.0, 348.0, 1120.0, 318.0, 35.0, 97.0, 831.0, 942.0, 280.0, 507.0, 681.0, 194.0, 365.0, 331.0, 464.0, 295.0, 868.0, 197.0, 249.0, 543.0, 573.0, 6.0, 393.0, 1187.0, 522.0, 608.0, 91.0, 144.0, 399.0, 438.0, 556.0, 71.0, 637.0, 470.0, 410.0, 102.0, 312.0, 390.0, 4756.0, 1590.0, 657.0, 5000.0, 1270.0, 345.0, 714.0, 1782.0, 2456.0, 4696.0, 1200.0, 469.0, 233.0, 2787.0, 236.0, 85.0, 100.0, 2159.0, 8.0, 532.0, 123.0, 424.0, 3023.0, 64.0, 751.0, 190.0, 22.0, 479.0,

In [0]:
from google.colab import files
import h5py
import pickle

weights = []
weights.append(seq2seq.emb_layer.get_weights())
weights.append(seq2seq.generator.get_weights())
weights.append(seq2seq.interpreter.get_weights())

def download_weights(weights):

    weight_file_name = "weights_wisebot_final3." +  \
                        str(MIN_QUOTE_LENGTH) + "." + \
                         str(MAX_QUOTE_LENGTH) + "." + \
                         str(MIN_TOKEN_FREQ) + ".h5"

    with h5py.File(weight_file_name, "w") as h5f:
        i = 0
        for weight in weights:
            group = h5f.create_group(str(i))
            i += 1
            j = 0
            for kernel in weight:
                group.create_dataset(str(j), data=kernel)
                j += 1
                
    h5f.close()
    files.download(weight_file_name)



words_file_name = "words_wisebot_final3." +  \
                        str(MIN_QUOTE_LENGTH) + "." + \
                         str(MAX_QUOTE_LENGTH) + "." + \
                         str(MIN_TOKEN_FREQ) + ".dict"

with open(words_file_name, "wb") as dict_file:
    pickle.dump(word_dict, dict_file)


begin_tokens_file_name = "begin_tokens." + str(MIN_QUOTE_LENGTH) + "." + \
                         str(MAX_QUOTE_LENGTH) + "." + \
                         str(MIN_TOKEN_FREQ) + ".arr"

with open(begin_tokens_file_name, "wb") as arr_file:
    pickle.dump(begin_tokens, arr_file)

download_weights(weights)
files.download(begin_tokens_file_name)
files.download(words_file_name)

In [0]:
print(begin_tokens)

[63.0, 77.0, 25.0, 140.0, 41.0, 61.0, 29.0, 66.0, 103.0, 67.0, 28.0, 70.0, 235.0, 11.0, 75.0, 39.0, 20.0, 217.0, 164.0, 369.0, 146.0, 170.0, 377.0, 166.0, 14.0, 46.0, 292.0, 206.0, 13.0, 350.0, 58.0, 86.0, 69.0, 256.0, 18.0, 366.0, 273.0, 74.0, 306.0, 261.0, 510.0, 351.0, 938.0, 55.0, 896.0, 173.0, 180.0, 159.0, 691.0, 254.0, 420.0, 129.0, 95.0, 309.0, 7.0, 104.0, 411.0, 222.0, 107.0, 59.0, 117.0, 1579.0, 536.0, 228.0, 333.0, 73.0, 1329.0, 82.0, 19.0, 27.0, 262.0, 472.0, 234.0, 33.0, 272.0, 44.0, 83.0, 1030.0, 944.0, 290.0, 153.0, 141.0, 348.0, 1120.0, 318.0, 35.0, 97.0, 831.0, 942.0, 280.0, 507.0, 681.0, 194.0, 365.0, 331.0, 464.0, 295.0, 868.0, 197.0, 249.0, 543.0, 573.0, 6.0, 393.0, 1187.0, 522.0, 608.0, 91.0, 144.0, 399.0, 438.0, 556.0, 71.0, 637.0, 470.0, 410.0, 102.0, 312.0, 390.0, 4756.0, 1590.0, 657.0, 5000.0, 1270.0, 345.0, 714.0, 1782.0, 2456.0, 4696.0, 1200.0, 469.0, 233.0, 2787.0, 236.0, 85.0, 100.0, 2159.0, 8.0, 532.0, 123.0, 424.0, 3023.0, 64.0, 751.0, 190.0, 22.0, 479.0,

In [0]:
print(tf.random.categorical(tf.ones((1,100)),100))

tf.Tensor(
[[41 82 85 80 13 14 74 84 89 21 52 22 89  9 76  3 75  3 16 58 95 93 93 73
  63  6 34 78 54 21 19 82 10 67 75 52 39 28 61 46 16 27 75 37 67 33 99 21
  45  8 91 17 75 27 98 84 46 76 33 93 21 92 25 79 45 57 65 28 33 82  7  0
  29 85 47 56 51 38 53 91  8 59 30 68 57 75 65  9 73  5 83 64 54 43 65 73
  49 50 63 50]], shape=(1, 100), dtype=int64)


In [0]:
files.download(begin_tokens_file_name)

In [0]:
begin_tokens_file_name = "begin_tokens." + str(MIN_QUOTE_LENGTH) + "." + \
                         str(MAX_QUOTE_LENGTH) + "." + \
                         str(MIN_TOKEN_FREQ) + ".arr"

with open(begin_tokens_file_name, "wb") as arr_file:
    pickle.dump(begin_tokens, arr_file)